In [8]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_sm
import en_core_web_lg
from word2number import w2n
from num2words import num2words

data = pd.read_csv('boardgames1.csv')
data.drop(data.columns.difference(['name', 'description', 'boardgamecategory']), 1, inplace = True) #Removing all irrelevant fields.
#data = data.sample(1000) # scale down data by a lot while testing
data["boardgamecategory"] = data["boardgamecategory"].apply(lambda s: s.replace("'", "").strip("[]"))

nlp = en_core_web_lg.load()

In [9]:
pd.options.display.max_colwidth = 190
data['description'][16177]

'8-28 is a press your luck card game. At the start of the game, each player gets one card they look at, then place face down in front of them. The cards have values ranging from 1 to 11. During the course of the game, the players may draw one random additional card per round to place face up in front of them. As long as any player still wants cards, gems are added to a pile. The round ends only when no one wants to take any more cards. Then, all cards are revealed and the two players whose cards are closest to a total of 8 and 28 respectively, without going over, share the gems among them.'

In [10]:
#Functions for pre processing text
def preprocess(text):
    l = list()
    for token in nlp(text):
        if (not token.is_stop and token.pos_ != 'PUNCT' and token.has_vector or token.pos_ == 'NUM'):  #Spacy currently considers '3', etc. to be stop words.
            if(token.pos_ == 'NUM'):
                l.append(token.text)
            else:
                l.append(token.lemma_)
    return l



In [11]:
test = preprocess('game. i.s for 3. players')
test #Looks good enough

['game', '3', 'player']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(tokenizer = preprocess)
X = vec.fit_transform(data["description"])

In [13]:
X.shape

(20000, 41439)

In [55]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import heapq
def search(query, num_desc = 10):
    '''
        Returns the app descriptions most similar 
        (euclidean distance) to the given 
        query as a Pandas DataFrame. The cosine similarity between
    '''
    # Process the query and retrieve the tfidf values of the terms
    processed_query = vec.transform([query])
    
    # Create a NearestNeighbors class with 10 return values and fit the model
    nbrs = NearestNeighbors(n_neighbors=num_desc).fit(X)
    
    # Process the query, returning the 10 nearest neighbors (distance and index)
    d,idx = nbrs.kneighbors(processed_query)
    similarities = []
    #print(idx)
    for i in idx[0]:
        similarities.append(nlp(str(' '.join(preprocess(data['description'][i])))).similarity(nlp(' '.join(preprocess(query)))))

    id = sorted(range(len(similarities)), key = lambda sub: similarities[sub])[-num_desc:]
    id.reverse()
    print(heapq.nlargest(num_desc, similarities))

    # Return the dataframe of the 10 closest neighbors
    return data.iloc[idx[0], 0:3]

In [56]:
search("I want a card game")

[0.891976565611905, 0.8918167334523603, 0.8891947208796324, 0.8855931906435223, 0.8852603540708922, 0.8851729991686306, 0.8783443941623279, 0.8642902569397176, 0.8511697300259623, 0.8506648226384232]


,name,boardgamecategory,description
16177,8-28,"Bluffing, Card Game","8-28 is a press your luck card game. At the start of the game, each player gets one card they look at, then place face down in front of them. The cards have values ranging from 1 to 11. ..."
11240,Go\/Stop,"Bluffing, Card Game",Go/Stop is a quick game that you try to obtain the highest point card (1-10). Everyone start with the set of identical cards (5 go cards and 1 stop card). For every point card flipped fr...
11314,Spiky Dastards,"Action / Dexterity, Party Game, Real-time","Spiky Dastards has 5 little figures that are so cute, you'll want to capture them all. But be careful, they're well protected! Split the deck of 60 cards into 2 piles and reveal the top ..."
14129,Star Plus,Card Game,"In Star Plus, players want to rid themselves of cards, but they can't just play anything that they want. No, the cards much be played in turn from 1-12 into the center of the table, then..."
6699,7 Ate 9,"Card Game, Math, Number",7 Ate 9 is a quick-playing card game in everyone wants to rid themselves of cards as quickly as possible &amp;mdash; but you can't play just any card you want! Each card features a large...
11292,Actionworks,"Card Game, Industry / Manufacturing",A fast-paced Industrial-Revolution-themed card game. Cards represent different kinds of actions in a factory. Players try to collect cards of their action type while using cards to cha...
7584,Escalation!,"Card Game, Humor","Players in Escalation! want to collect as few cards as possible. Each player starts with a hand of six cards from a deck that contains cards numbered 1-13, three jokers that can be worth..."
16840,The Walking Dead Card Game,"Card Game, Movies / TV / Radio theme, Zombies","The Walking Dead Card Game, based on Wolfgang Kramer's 6 nimmt!, features the same basic gameplay as that card game while adding six character cards and two modes of play: Survival and H..."
9842,Olympicards,"Card Game, Sports","Olympicards has the player athletes compete in nine different events, with competition taking place via card play. Each sport has its own rules for playing cards, so sometimes you'll wan..."
10715,The Bark Side,Card Game,"In The Bark Side, you can be a bad doggie as much as you want &amp;mdash; as long as you're not the one caught in the end. The game consists of a sixty-card deck, with five copies each ..."


In [16]:
def keywords(text, n=10):
    '''
    Returns a list with the n (default value: 10) most salient keywords from the specified text, 
    as measured by their tf-idf value relative to the collection of app descriptions.
    '''   
    # Process the query using the TfidfVectorizer 
    # and place the contents into a numpy.array
    processed_query = vec.transform([text]).toarray()

    # Get yet another numpy array of the descriptions words.
    words = np.array(vec.get_feature_names())
    
    # Get the indexes and sort them based on the tfidf value.
    sorted_tfidf = np.argsort(processed_query[0])[::-1]    
    return words[sorted_tfidf[:n]]

In [17]:
#insert index of sought after row
#print(keywords(data["description"][1676]))

In [18]:
# Extract the idf value from the TfidfVectorizor and 
# connect it to the feature names present.
terms = np.column_stack([vec.idf_, vec.get_feature_names()])

# Create a DataFrame with column headers
terms = pd.DataFrame(terms, columns=['value','term'])

# Sort the values by BOTH columns, 
# where the idf are ordered in a ascending manner  (1 -> 2)
# and the terms are ordered in an ascending manner (a -> b)
# place the ordered terms into terms by inplace=True.
terms.sort_values(by=['value', 'term'], ascending=(True,True), inplace=True)

print(terms[:10])
print(terms[-10:])

                    value     term
16955  1.1363853187072985     game
28895  1.2367855035837183   player
26967  1.7171825420604985      one
8598    1.728824356953138     card
28885   1.741757369925364     play
8       1.763297806989476        &
38189  1.9964667751034817     turn
0      10.210390370726225    1796
1      10.210390370726225      be
2      10.210390370726225  enough
                  value     term
41366  9.80492526261806      zun
41367  9.80492526261806     zuni
41382  9.80492526261806      ~10
41394  9.80492526261806  édition
41398  9.80492526261806        а
41400  9.80492526261806      для
41401  9.80492526261806        и
41403  9.80492526261806       на
41407  9.80492526261806        ―
41438  9.80492526261806        ？


In [19]:
"""Creating a list of all categories present in the column 'boardgamecategory'"""
categories_l = [l.strip('[]') for l in data['boardgamecategory']]
categories = set()
for s in categories_l:
    s = s.split(",")
    for cat in s:
        categories.add(cat.replace("'", "").strip())

real_categories = [cat.strip() for cat in open("categories.txt")]
categories = categories and real_categories

#Modify dataframe to include hot 1s for matching categories
for cat in categories:
    l = []
    for index, row in data.iterrows():
        if cat in row['boardgamecategory']:
            l.append(1)
        else:
            l.append(0)
    data[cat] = l

In [20]:
data['Prehistoric']

0        0
1        0
2        0
3        0
4        0
        ..
19995    0
19996    0
19997    0
19998    0
19999    0
Name: Prehistoric, Length: 20000, dtype: int64

In [21]:
def most_similar(x, n = 10):
    numpyx = np.asarray([x])
    m_s = nlp.vocab.vectors.most_similar(numpyx, n=n)
    ret = [ nlp.vocab[key] for key in m_s[0][0] ]
    return ret

In [22]:
from spacy.tokens import Doc
sent1 = nlp("Solving murder. Detectives")
sent2 = nlp("Childrens Game, Deduction")
sent3 = nlp("Deduction, Dice, Murder/Mystery")

sim1 = sent1.similarity(sent2) #Similarity uses the cosine similarity using word vectors
sim2 = sent1.similarity(sent3) 

In [23]:
from sklearn.base import TransformerMixin, BaseEstimator
class emb_transformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("init called\n")

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        print("transform called\n")
        return np.array([nlp(' '.join(preprocess(sent))).vector for sent in X])


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mr_pipe = Pipeline([('emb_t', emb_transformer()), ('clf', OneVsRestClassifier(DecisionTreeClassifier()))])

train, test = train_test_split(data, test_size = 0.33)

X_train = train.description
X_test = test.description

init called



In [25]:
#Classifier for seeing which categories are easiest to recognize. Takes VERY long.
"""
for category in categories:
    print('category {}'.format(category))
    mr_pipe.fit(X_train, train[category])
    # classifier.fit(X_train, train[category])
    pred = classifier.predict(X_test)
    print("Accuracy: {}".format(accuracy_score(test[category], pred)))
"""

'\nfor category in categories:\n    print(\'category {}\'.format(category))\n    mr_pipe.fit(X_train, train[category])\n    # classifier.fit(X_train, train[category])\n    pred = classifier.predict(X_test)\n    print("Accuracy: {}".format(accuracy_score(test[category], pred)))\n'

In [26]:
processed = []
for row in data['description']:
    processed.append(nlp(' '.join(preprocess(row))))


In [27]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(tokenizer = preprocess)
count_vec.fit_transform(data["description"])


<20000x41439 sparse matrix of type '<class 'numpy.int64'>'
	with 1317480 stored elements in Compressed Sparse Row format>

In [28]:
def smooth_inverse(sent, emb_size = 300, a = 1e-3):
    sent_set = []
    vs = np.zeros(emb_size)
    sent_length = len(sent)
    #print(sent[0])
    for word in sent:
        try:
            a_value = a / (a + count_vec.vocabulary_[word]) # word_count
            # a_value = a / (a + vec.idf_[vec.vocabulary_[word]]) # TF_IDF
        except:
            a_value = a / (a + 1) # Which value to put if the word is not in corpus? 
        vs = np.add(vs, np.multiply(a_value, word.vector))

        vs = np.divide(vs, sent_length)
        sent_set.append(vs)
    return sent_set

#https://blogs.nlmatics.com/nlp/sentence-embeddings/2020/08/07/Smooth-Inverse-Frequency-Frequency-(SIF)-Embeddings-in-Golang.html
processed_SIF = []
for sent in processed:
    processed_SIF.append(sum(vector for vector in smooth_inverse(sent)) / len(sent))




In [29]:
from sklearn.decomposition import TruncatedSVD
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import numpy
import math
N_pc_rem = 1
pca = TruncatedSVD(N_pc_rem, n_iter = 7, random_state = 0)
sent_emb = np.array(processed_SIF)
pca.fit(sent_emb)

# Print which words are close to the first principle component
print(' '.join(w.text for w in most_similar(pca.components_[0], 1000)))

if N_pc_rem == 1:
    updated_embs = sent_emb - sent_emb.dot(pca.components_.transpose()) * pca.components_
else:
    updated_embs = sent_emb - sent_emb.dot(pca.components_.transpose()).dot(pca.components_)

One ONE oNE onE one ONe OnE EVen even EVEN Even WAY Way way wAy but BuT But buT BUT BUt bUt bUT that ThAT thAT tHAT THat tHat ThaT thaT tHaT THAT THAt thAt ThAt That could COuld COULD Could IT It it iT İt there THere There THERE Make MAKE mAke make MAke theY THey THEY They they RATHER rather Rather Because because BECAUSE BEcause becAuse WHEN When when WHen Same same SAme SAME SAMe ENOUGH enough Enough jUst JUst jUST JUSt JUST Just JuSt just iF IF if If İf so So SO sO WOuld Would would wOuld WOULD Kind KIND kind well WELL Well WEll Thing THing thing THING GOING GOing going Going though Though THough THOUGH where WHERe WHere Where WHERE Once ONce ONCE once PUT Put put might Might MIGHT MIght able Able ABLE whatever WHatever WHATEVER Whatever WhAT what WHAT WHAt What wHat wHaT whaT wHAT WHat whAt WhAt WhaT whAT HoW how HOw HOW hOw hoW hOW How GET GEt gEt gET get GeT Get ANOTHER ANother Another another STILL STill still Still Better betteR BETTER better actually ACTUALLY Actually ACtually

In [57]:
#Only using cosine similarity gives rather bad results with longer sentences as words with no real meaning counts as much as more relevant ones.
#Solution, smooth inverse frequency!

def search_emb(query, num_desc = 10):
    """
    search_emb utilizes the doc.similarity function from spacy.
    It utilizes the cosince similarity between the word embeddings of the two sentences
    """

    processed_query = nlp(' '.join(preprocess(query)))
    #print(processed_query)
    similarities = []
    for row in processed:
        similarities.append(processed_query.similarity(row))

    idx = sorted(range(len(similarities)), key = lambda sub: similarities[sub])[-num_desc:]
    idx.reverse()

    print(heapq.nlargest(num_desc, similarities))
    return data.iloc[idx, 0:3]

In [58]:
def search_emb_2(query, num_desc = 10):
    """
    search_emb_2 utilizes the first part of SIF (adding a weight to each embedding based on their word count).
    This reduces the importance of words which are used more often such as "I" "can" "and".
    """
    processed_query = nlp(' '.join(preprocess(query)))
    obj1_SIF = sum(vector for vector in smooth_inverse(processed_query)) / len(processed_query)

    similarities = []
    for row in processed_SIF:
        similarities.append(cosine_similarity(obj1_SIF.reshape(1, -1), row.reshape(1, -1))[0][0])

    idx = sorted(range(len(similarities)), key = lambda sub: similarities[sub])[-num_desc:]
    idx.reverse()
    print(heapq.nlargest(num_desc, similarities))
    return data.iloc[idx, 0:3]

In [59]:
def search_emb_3(query, num_desc = 10):

    """
    search_emb_3 utilizes the first and second part of SIF.adding a weight to each embedding based on their word count
    and removing the first principle component.
    """
    processed_query = nlp(' '.join(preprocess(query)))

    obj1_SIF = sum(vector for vector in smooth_inverse(processed_query)) / len(processed_query)
    if N_pc_rem == 1:
        obj1_SIF = obj1_SIF - obj1_SIF.dot(pca.components_.transpose()) * pca.components_
    else:
        obj1_SIF = obj1_SIF - obj1_SIF.dot(pca.components_.transpose()).dot(pca.components_)

    similarities = []
    for row in updated_embs:
        similarities.append(cosine_similarity(obj1_SIF.reshape(1, -1), row.reshape(1, -1))[0][0])

    idx = sorted(range(len(similarities)), key = lambda sub: similarities[sub])[-num_desc:]
    idx.reverse()
    print(heapq.nlargest(num_desc, similarities))
    return data.iloc[idx, 0:3]

In [81]:
search("I am looking for a board game about space, where players take turns deciding their actions. The game should involve strategy and negotiation. The number of players should range from 2-4.")

[0.9281944646361496, 0.9266791010474814, 0.9263521943836778, 0.9231815772788711, 0.9125682343507695, 0.9122952427361442, 0.9040733206705154, 0.9026389795862912, 0.8847076722684155, 0.8487108665031285]


,name,boardgamecategory,description
17198,Word Power,"Card Game, Dice, Word Game","Players take turns rolling and moving around a circular board (a la Monopoly) in an attempt to &amp;quot;publish&amp;quot; 10 books or obtain $100,000. Each player starts with $20,000 a..."
18534,Archimedes,Abstract Strategy,This is a strategy game for 2-3 players. It takes place on a 6 sided game board with pieces moving on the triangles. Each player has 4 pawns in the form of dice. The triangular fields of...
18869,Winkeladvokat,Abstract Strategy,Winkeladvokat is an abstract game for 2 to 4 players. Re-implemented as: Cabale
8060,Mana,Abstract Strategy,Abstract strategy game for 2 players. Players try to capture the opponent's Damyo with their own Damyo or one of their 5 Ronins. Game contains one 6&amp;times;6 Leather Board (12 single...
12500,Kings' Struggle,"Card Game, Medieval, Negotiation","Welcome to Kings' Struggle, a negotiation-focused card game with elements of trick-taking and set collection. The game is mechanically simple, with each player playing just one card per ..."
17300,Colorio,Abstract Strategy,"Colorio starts with players using plastic tokens to cover the colored spaces on the 5x5 game board. On each turn, a player must remove three tokens from the board, then either remove all..."
3441,Rondo,"Abstract Strategy, Number","In the abstract strategy game Rondo, players try to score as many points as possible by placing tokens on numbered spaces on the game board, with the board consisting of two concentric r..."
491,Tokaido,Travel,"In Tokaido, each player is a traveler crossing the &amp;quot;East sea road&amp;quot;, one of the most magnificent roads of Japan. While traveling, you will meet people, taste fine meals,..."
9153,Futboard,Sports,"Check the video! https://www.facebook.com/futboardgame/videos/143893042968456/ The ultimate tactical Soccer Board Game from Brazil. For everyone who doesn't like the sport, but is in th..."
19950,The Ugly Duckling Game,Childrens Game,"The most basic of children's games. Spin the spinner and move your pawn the number of spaces. If you land on a draw a card space, you draw a card and move forward the number of spaces ..."


In [75]:
search_emb("I want a game surrounding fictional litterature. I would prefer one involving elements of horror or lovecraftian themes. The game should take place in a mysterious city where creatures lurk around every corner. The game time should be about 4 hours. My board game club consists of 5 people and the game should therefore contain at least that many.")

[0.929711298472263, 0.9271717866393533, 0.9253764377972572, 0.9231193104732136, 0.9229101898397469, 0.9228614841318387, 0.9220134080483673, 0.9207456837499431, 0.9198028808957559, 0.9196671802905498]


,name,boardgamecategory,description
14594,Miskatonic Madness,"Horror, Print & Play","This game is set in H.P. Lovecraft's fictional Miskatonic University. The two players take on the roles of either the Cultists out to wake one of the elder gods, or the Investigators try..."
12296,Meeple Quest,"Adventure, Exploration, Fantasy, Fighting, Humor",This is a fantasy boardgame with true dungeon exploring mechanics. It has dinamically generated dungeons which will be completely different from game to game. In it we are little meeples...
10836,Zombie Cinema,"Horror, Zombies","Zombie Cinema is a boardgame for 3-6 players, Ages 12 and up. One game takes 2-3 hours, all told. Learning the game takes ten minutes. The game is a cooperative storytelling exercise whe..."
15730,Mees,"Adventure, Card Game, Dice, Novel-based","Adventure game &amp;quot;The man who knew snakewords&amp;quot; are on a very hazy border of card games and board games. The game has 25 landscape cards, which create the game of risks an..."
2511,Adventure Games The Dungeon,"KOSMOS, 999 Games, Zvezda","Explore places, combine items, and experience stories in Adventure Games, a series of co-operative games from German publisher KOSMOS. In each of these titles, players are presented with..."
15889,Heroes of Might & Magic IV Collectible Card & Tile Game,"Card Game, Collectible Components, Exploration, Fantasy, Medieval, Mythology",The Heroes of Might &amp;amp; Magic IV Collectible Card &amp;amp; Tile Game allows you to play all of the exciting creatures and characters found in the classic video game series by Ubis...
13835,Dragonfire,"Adventure, Fantasy","Dragonfire was an attempt by Target/Heartbreaker to entice young adults into roleplaying, by means of devising a crossover between a roleplaying game and a fantasy miniatures boardgame s..."
4928,The Sorcerer's Cave,"Card Game, Exploration, Fantasy","A fantasy-based game, using cards to create an underground lair that players explore -&amp;ndash; cooperatively or competitively -- using bands of characters, in an attempt to leave with..."
12256,Hemloch Vault of Darkness,"Card Game, Fantasy","Hemloch is a peculiar city; its nights and days last for decades, and control of the city swings wildly during the weeks leading up to the change. The Great Houses, using Minions as ark..."
11187,Dreamwars,"Adventure, Fantasy, Horror, Miniatures, Science Fiction","From the publisher: Dreamwars is a 1 to 8 players boardgame set in an original steampunk fantasy setting with a strong horror vibe, where technology coexists side by side with the ancie..."


In [80]:
search_emb_2("I want a game surrounding fictional litterature. I would prefer one involving elements of horror or lovecraftian themes. The game should take place in a mysterious city where creatures lurk around every corner. The game time should be about 4 hours. My board game club consists of 5 people and the game should therefore contain at least that many.")

[0.9297734154943217, 0.927213197045355, 0.925381415916226, 0.9232471274646198, 0.9228373236239817, 0.9228246248061416, 0.9219602010866329, 0.9207907243434806, 0.9197634597660574, 0.9196410174273439]


,name,boardgamecategory,description
14594,Miskatonic Madness,"Horror, Print & Play","This game is set in H.P. Lovecraft's fictional Miskatonic University. The two players take on the roles of either the Cultists out to wake one of the elder gods, or the Investigators try..."
12296,Meeple Quest,"Adventure, Exploration, Fantasy, Fighting, Humor",This is a fantasy boardgame with true dungeon exploring mechanics. It has dinamically generated dungeons which will be completely different from game to game. In it we are little meeples...
10836,Zombie Cinema,"Horror, Zombies","Zombie Cinema is a boardgame for 3-6 players, Ages 12 and up. One game takes 2-3 hours, all told. Learning the game takes ten minutes. The game is a cooperative storytelling exercise whe..."
15730,Mees,"Adventure, Card Game, Dice, Novel-based","Adventure game &amp;quot;The man who knew snakewords&amp;quot; are on a very hazy border of card games and board games. The game has 25 landscape cards, which create the game of risks an..."
2511,Adventure Games The Dungeon,"KOSMOS, 999 Games, Zvezda","Explore places, combine items, and experience stories in Adventure Games, a series of co-operative games from German publisher KOSMOS. In each of these titles, players are presented with..."
15889,Heroes of Might & Magic IV Collectible Card & Tile Game,"Card Game, Collectible Components, Exploration, Fantasy, Medieval, Mythology",The Heroes of Might &amp;amp; Magic IV Collectible Card &amp;amp; Tile Game allows you to play all of the exciting creatures and characters found in the classic video game series by Ubis...
13835,Dragonfire,"Adventure, Fantasy","Dragonfire was an attempt by Target/Heartbreaker to entice young adults into roleplaying, by means of devising a crossover between a roleplaying game and a fantasy miniatures boardgame s..."
4928,The Sorcerer's Cave,"Card Game, Exploration, Fantasy","A fantasy-based game, using cards to create an underground lair that players explore -&amp;ndash; cooperatively or competitively -- using bands of characters, in an attempt to leave with..."
12256,Hemloch Vault of Darkness,"Card Game, Fantasy","Hemloch is a peculiar city; its nights and days last for decades, and control of the city swings wildly during the weeks leading up to the change. The Great Houses, using Minions as ark..."
10123,Ghoulash,"Exploration, Fantasy, Horror, Maze, Print & Play","Ghoulash is a two-player strategy game played entirely on paper, using specially designed Game Charts. Think of Ghoulash as a miniature RPG in which each player is the other&amp;rsquo;s ..."


In [70]:
search_emb_3("I am interested in history, in particular, the history of famous wars such as world war 1 and world war 2.")

[0.686434116076394, 0.666483262600067, 0.6404878743219872, 0.6386173277625903, 0.6359292739114899, 0.6358728828876444, 0.6328409893045794, 0.6283285942319716, 0.6115431989555778, 0.6058230352951807]


,name,boardgamecategory,description
19779,The War to End All Wars World War 1,"Dice, Economic, Educational, Wargame, World War I","The War to End All Wars is a World War I territorial strategy game. A variant of the Axis &amp;amp; Allies board game system, it offers an easy to play D10 based combat system and territ..."
4621,World at War The Untold Stories,"Modern Warfare, Wargame","World at War: The Untold Stories recounts the struggle of Warsaw Pact and NATO allies previously not covered in World at War modules. Included are the battles of the Canadians, East Germ..."
8945,The Great Pacific War,"Wargame, World War II",Description from the publisher: Great Pacific War is Avalanche Press' strategic-level game of the War in the Pacific. The game covers the entire war from December 1941 through 1946 and ...
15485,Pearl Harbor,"Wargame, World War II","&amp;quot;The War Against Japan, 1941-1945.&amp;quot; &amp;quot;A game-simulation of the Pacific and Asian theaters of World War II, at the strategic level. Simulated are the major poli..."
13540,Twilight Squabble,"Card Game, Modern Warfare, Political, Wargame","Relive the entire Cold War in ten minutes with Twilight Squabble! In 1947, the superpowers of the United States of America and the Soviet Union began a new sort of conflict, a struggle ..."
4414,Grand Illusion Mirage of Glory,"Wargame, World War I","The German invasion of France and Belgium in 1914, the opening campaign of World War One, is one of the most dramatic in history: the famed German Schlieffen Plan, the infamous French Pl..."
12087,Viva Espa\u00f1a,"Civil War, Wargame","VIVA ESPA&amp;Ntilde;A is a wargame,or historical simulation, that combines gaming techniques with historical data to duplicate conditions as they were at the time the actual events were..."
5090,Empires in America (second edition),"Age of Reason, American Indian Wars, Wargame","Empires in America (EIA) is a solitaire States of Siege&amp;trade; Series game of the conflict in North America between the French and British in the mid-18th century. This war, known lo..."
8618,Patton in Flames,"Wargame, World War II",It's 1945; Germany and Japan are going down in flames. But the Western Allies and the Soviets will soon find themselves at war... It's 1948; rising tensions between the Western Allies ...
10021,Trafalgar,"Dice, Miniatures, Napoleonic, Nautical, Wargame",Trafalgar Naval Warfare in the Age of Sail (1795-1815) by Warhammer Historical Wargames Ltd is a set of tabletop wargame rules set during the time of the Napoleonic Wars. Fleetlists are ...
